# Proyecto 1: Entendimiento de los datos con PySpark (Aeropuerto).

## Paso #1: Setup

En esta seccion se hara el setup de la tarea, incluyendo librerias y estableciendo todo los prerequisitos para la correcta ejecucion de lo esperado en la actividad.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions
from pyspark.sql.types import StructType
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql.types import FloatType, StringType, IntegerType, DateType
from pyspark.sql.functions import udf, col, length, isnan, when, count, regexp_replace, abs
import pyspark.sql.functions as f
import os 
from datetime import datetime
from pyspark.sql import types as t
from pandas_profiling import ProfileReport
import matplotlib.pyplot as plt
import numpy as np

path_jar_driver = 'C:\Program Files (x86)\MySQL\Connector J 8.0\mysql-connector-java-8.0.28.jar'

conf=SparkConf() \
    .set('spark.driver.extraClassPath', path_jar_driver)

spark_context = SparkContext(conf=conf)
sql_context = SQLContext(spark_context)
spark = sql_context.sparkSession

db_connection_string = 'jdbc:mysql://157.253.236.116:8080/ProyectoTransaccional'
db_user = 'Estudiante_103_202413'
db_psswd = 'MISO_aabb1122'

df_aeropuertos = spark.read.format('jdbc')\
    .option('url', db_connection_string) \
    .option('dbtable', '''(SELECT * FROM aeropuertosCopia ) AS Compatible''') \
    .option('user', db_user) \
    .option('password', db_psswd) \
    .option('driver', 'com.mysql.cj.jdbc.Driver') \
    .load()

print("La tabla extraida de la fuente de datos se ve de la siguiente manera: ")
df_aeropuertos.show()

C:\Users\estudiante\anaconda3\envs\Tutoriales\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\estudiante\anaconda3\envs\Tutoriales\lib\site-packages\pyspark\sql\context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


La tabla extraida de la fuente de datos se ve de la siguiente manera: 
+-----+----+--------------------+--------------------+------------+----------+-------+--------+--------------------+--------------------+--------------+-----------+-------+-----------+---------+----------+------------------+--------------+-----+-----------+--------------------+----------------+-------------+
|sigla|iata|              nombre|           municipio|departamento| categoria|latitud|longitud|         propietario|          explotador|longitud_pista|ancho_pista|   pbmo|orientacion|elevacion|resolucion|fecha_construccion|fecha_vigencia|clase|       tipo|numero_vuelos_origen|gcd_departamento|gcd_municipio|
+-----+----+--------------------+--------------------+------------+----------+-------+--------+--------------------+--------------------+--------------+-----------+-------+-----------+---------+----------+------------------+--------------+-----+-----------+--------------------+----------------+-------------+

## Paso #2: Entendimiento de los Datos.
En este paso se espera poder comprender mejor la informacion que se tienen en la estructura de datos.

El negocio entrego un diccionario de datos que se puede observar en el siguiente [enlace](../Diccionario%20IV.xlsx)

In [2]:
print("En la siguiente impresion se puede ver los tipos de datos de la tabla extraida")
for field in df_aeropuertos.schema.fields:
    print(field.name +" , "+str(field.dataType))

En la siguiente impresion se puede ver los tipos de datos de la tabla extraida
sigla , StringType
iata , StringType
nombre , StringType
municipio , StringType
departamento , StringType
categoria , StringType
latitud , IntegerType
longitud , StringType
propietario , StringType
explotador , StringType
longitud_pista , IntegerType
ancho_pista , DoubleType
pbmo , StringType
orientacion , StringType
elevacion , StringType
resolucion , StringType
fecha_construccion , StringType
fecha_vigencia , StringType
clase , StringType
tipo , StringType
numero_vuelos_origen , DoubleType
gcd_departamento , IntegerType
gcd_municipio , IntegerType


In [3]:
print('La cantidad de registros obtenidos es de: ' + str(df_aeropuertos.count()) )

La cantidad de registros obtenidos es de: 547


#### Significado de una Fila:

Usando los datos presentados anteriormente se puede concluir que una Fila de esta tabla representa un resumen tecnico de la informacion de los aeropuertos de los que se tienen datosen Colombia, provee informacion muy tecnica y especializada de la aviacion (como el IATA, la categoria, el pbmo y la clase), pero tambien ofrece informacion que personas comunes puede consumir como nombre, municipio, latitud, longitud, etc.

### Analisis de Campos numericos

In [4]:
df_aeropuertos_num = df_aeropuertos.select('latitud', 'longitud_pista', 'ancho_pista', "numero_vuelos_origen", 'gcd_departamento', 'gcd_municipio')
df_aeropuertos_num.summary().show()

df_aeropuertos_ancho_pista = df_aeropuertos.groupby('ancho_pista').count().show()
df_aeropuertos_longitud_pista = df_aeropuertos.groupby('longitud_pista').count().show()
df_aeropuertos_latitud = df_aeropuertos.groupby('latitud').count().show()

+-------+-------+--------------+------------------+--------------------+------------------+-----------------+
|summary|latitud|longitud_pista|       ancho_pista|numero_vuelos_origen|  gcd_departamento|    gcd_municipio|
+-------+-------+--------------+------------------+--------------------+------------------+-----------------+
|  count|    547|           547|               547|                 547|               547|              547|
|   mean|  500.0|         250.0|6.7714808043875685|   -1495.54844606947|62.029250457038394|62342.82998171847|
| stddev|    0.0|           0.0|2.8868893676039455|  46097.478663370515| 28.13857428200601|28082.87921613649|
|    min|    500|           250|               0.0|          -1028334.0|                 5|             5001|
|    25%|    500|           250|               8.0|               -12.0|                47|            47058|
|    50%|    500|           250|               8.0|                 0.0|                73|            73001|
|    75%| 

Con este analisis numericos podemos desprender los siguientes aprendizajes:

- La longitud de pista es de 250 siempre, ya que la desviacion estandar es 0.0
- Existen 84 registros con 0.0 de ancho de pista.
- Todos los registros tienen latitud 500.
- El numero de vuelos cuyo origen es el aeropuerto en cuestion tiene un deviacion estandar en donde los valores del percentil 75% es 6 y el maximo es 207439.
- Existen numeros de vuelos cuyo origen es el aeropuerto en cuestion negativos, esto tal vez se deba a algun tipo de error.
- Los campos de `gcd_departamento` y `gcd_municipio` poseen una sana distribucion y relacion entre ellos lo que puede indicar que los valores estan correctos.


#### Analisis descriptivo de los datos:

In [5]:
df_aeropuertos_desc = df_aeropuertos.select('sigla', 'iata', 'nombre', 'municipio', 'departamento', 'categoria', 'longitud', 'propietario', 'explotador', 'pbmo', 'orientacion', 'elevacion', 'resolucion', 'fecha_construccion', 'fecha_vigencia', 'clase', 'tipo')
df_aeropuertos_desc.show(5)

df_aeropuertos_categoria = df_aeropuertos.groupby('categoria').count().show()
df_aeropuertos_tipo = df_aeropuertos.groupby('tipo').count().show()
df_aeropuertos_clase = df_aeropuertos.groupby('clase').count().show()

+-----+----+--------------------+--------------------+------------+----------+--------+-----------------+--------------------+------+-----------+---------+----------+------------------+--------------+-----+--------+
|sigla|iata|              nombre|           municipio|departamento| categoria|longitud|      propietario|          explotador|  pbmo|orientacion|elevacion|resolucion|fecha_construccion|fecha_vigencia|clase|    tipo|
+-----+----+--------------------+--------------------+------------+----------+--------+-----------------+--------------------+------+-----------+---------+----------+------------------+--------------+-----+--------+
|  9cg|    |san jose del ariporo|      Paz de Ariporo|    casanare|AerÃ³dromo|-70.0128|        MUNICIPIO|           MUNICIPIO|2000.0|           |    318.0|  4541,000|        2006-10-27|    2016-11-07|   1A|PÃºblico|
|  mii|    |                mani|               ManÃ­|    casanare|AerÃ³dromo|        |                 |                    |      |   

Con el analisis descriptivo anterior se pueden extraer las siguientes conclusiones:

- Existen muchos registros con cadenas de caracteres vacias, es decir `""`.
- Existen errores de codificacion UTF, ya que algunos caracteres especiales del español como letras con acentos tal vez necesiten tratamiento para ser mas explicitos.

## Paso #3: Analisis de Calidad de los Datos.
En este paso se espera identificar areas de oportunidad en los datos suministrados.

#### Unicidad y validez:

En esta seccion se verificara la existencia o no de datos duplicados.

In [6]:
unicos_siglas = df_aeropuertos.select("sigla").distinct().count()
unicos_iata = df_aeropuertos.select("iata").distinct().count()
totales = df_aeropuertos.count()

print('Existen ' + str(unicos_siglas) + ' registros unicos por sigla, de un total de ' + str(totales) )
print('Existen ' + str(unicos_iata) + ' registros unicos por IATA, de un total de ' + str(totales) )
print ("Porcentaje de Valores de Siglas Duplicados: " + str(((totales - unicos_siglas ) / totales )* 100) + "%" )
print ("Porcentaje de Valores de IATA Duplicados: " + str(((totales - unicos_iata ) / totales )* 100) + "%" )


Existen 443 registros unicos por sigla, de un total de 547
Existen 53 registros unicos por IATA, de un total de 547
Porcentaje de Valores de Siglas Duplicados: 19.012797074954296%
Porcentaje de Valores de IATA Duplicados: 90.31078610603291%


**Duplicados Logicos**

Para los duplicados logicos se utilizo unicamente los camposde `siglas` y `iata`, campos que representan identificadores unicos para los aeropuertos uno asignado a nivel nacional y el otro a nivel internacional.

Con el analisis anterior podemos ver que usando los identificadores principales de los registros que son las siglas y el campo de IATA, se pueden ver que existen maximo 443 registros unicos, quedando un porcentaje de al menos 19% de datos duplicados.

Existe la posibilidad que diversos aeropuertos no cuenten con un vampo de `iata` dado que no tenga vuelos internacionales, por eso solo se considero el campo de siglas para ete analisis.

In [7]:
duplicados_totales = df_aeropuertos.count() - df_aeropuertos.distinct().count()
print("La cantidad de registros duplicados totales es de " + str(duplicados_totales))

print ("Porcentaje de Valores de Duplicados totale: " + str((duplicados_totales / totales )* 100) + "%" )

La cantidad de registros duplicados totales es de 103
Porcentaje de Valores de Duplicados totale: 18.82998171846435%


**Duplicados Totales**

En el analisis anterior podemos ver que a nivel de duplicados totales existen 103 registros que son duplicados totales correspondiente a un 18.8 porciento del total de registros.

**Despues de todo el analisis a nivel de Unicidad se puede considerar que los datos posee un nivel de calidad medio en este apartados, ya que con valores repetidos de entre el 18 y 19 por ciento, deberia el negocio revisar sus fuentes de datos y encontrar la manera de distinguir entre los registro con la misma informacion**

#### Completitud y validez:

In [8]:
def contar_vacios(df):
    resultados = []
    for c in df.columns:
        vacios = df.filter(df[c].isNull()).count()
        if vacios!=0:
            print('número de vacíos para columna '+c+': '+str( vacios ))
            resultados.append(vacios)
        else:
            print('La columna '+c+' no presenta filas vacias.')
    return resultados

columnas_vacias = contar_vacios(df_aeropuertos)

La columna sigla no presenta filas vacias.
La columna iata no presenta filas vacias.
La columna nombre no presenta filas vacias.
La columna municipio no presenta filas vacias.
La columna departamento no presenta filas vacias.
La columna categoria no presenta filas vacias.
La columna latitud no presenta filas vacias.
La columna longitud no presenta filas vacias.
La columna propietario no presenta filas vacias.
La columna explotador no presenta filas vacias.
La columna longitud_pista no presenta filas vacias.
La columna ancho_pista no presenta filas vacias.
La columna pbmo no presenta filas vacias.
La columna orientacion no presenta filas vacias.
La columna elevacion no presenta filas vacias.
La columna resolucion no presenta filas vacias.
La columna fecha_construccion no presenta filas vacias.
La columna fecha_vigencia no presenta filas vacias.
La columna clase no presenta filas vacias.
La columna tipo no presenta filas vacias.
La columna numero_vuelos_origen no presenta filas vacias.
L

In [9]:
df_aeropuertos.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_aeropuertos.columns]
   ).show()

+-----+----+------+---------+------------+---------+-------+--------+-----------+----------+--------------+-----------+----+-----------+---------+----------+------------------+--------------+-----+----+--------------------+----------------+-------------+
|sigla|iata|nombre|municipio|departamento|categoria|latitud|longitud|propietario|explotador|longitud_pista|ancho_pista|pbmo|orientacion|elevacion|resolucion|fecha_construccion|fecha_vigencia|clase|tipo|numero_vuelos_origen|gcd_departamento|gcd_municipio|
+-----+----+------+---------+------------+---------+-------+--------+-----------+----------+--------------+-----------+----+-----------+---------+----------+------------------+--------------+-----+----+--------------------+----------------+-------------+
|    0|   0|     0|        0|           0|        0|      0|       0|          0|         0|             0|          0|   0|          0|        0|         0|                 0|             0|    0|   0|                   0|            

Con los dos bloques anteriores podemos ver que en los registros extraidos no existe datos vacios o nulos, pero gracias a la inspeccion visual y elanalisis de datos descriptivos que se hizo en el paso anterior se puede determinar que la existencia de datos con cadena de caracteres vacias (es decir "") pueden enmascarar problemas de completitud que el negocio seguramente querrar atender.

In [10]:
df_aeropuertos.select([count(when(col(c) == "",c)).alias(c) for c in df_aeropuertos.columns]
   ).show()

+-----+----+------+---------+------------+---------+-------+--------+-----------+----------+--------------+-----------+----+-----------+---------+----------+------------------+--------------+-----+----+--------------------+----------------+-------------+
|sigla|iata|nombre|municipio|departamento|categoria|latitud|longitud|propietario|explotador|longitud_pista|ancho_pista|pbmo|orientacion|elevacion|resolucion|fecha_construccion|fecha_vigencia|clase|tipo|numero_vuelos_origen|gcd_departamento|gcd_municipio|
+-----+----+------+---------+------------+---------+-------+--------+-----------+----------+--------------+-----------+----+-----------+---------+----------+------------------+--------------+-----+----+--------------------+----------------+-------------+
|    0| 482|     0|        0|           0|       41|      0|     140|        141|       141|             0|          0| 194|        547|      141|       147|               383|           460|  141| 141|                   0|            

Con el bloque anterior se puede comprobar que existen hasta 547 registros con valores de cadenas de caracteres vacios (para la columna de orientacion) que pueden afecta como se interpreta la completitud de los datos.

In [11]:
def cardinalidad(df):
    resultado = {}
    baja_cardinalaidad = {}
    for c in df.columns:
        cardinalidad = df.select(col(c)).distinct().count()
        if cardinalidad>=df.count()*0.5:
            resultado[c] = cardinalidad
        else:
            baja_cardinalaidad[c] = cardinalidad
    return resultado, baja_cardinalaidad

columnas_alta_cardinalidad, columnas_baja_cardinalidad  = cardinalidad(df_aeropuertos)
print(" Las columnas con Alta cardinalidad son: " + str(columnas_alta_cardinalidad) )
print(" Las columnas con Baja cardinalidad son: " + str(columnas_baja_cardinalidad) )

 Las columnas con Alta cardinalidad son: {'sigla': 443, 'nombre': 413, 'longitud': 324}
 Las columnas con Baja cardinalidad son: {'iata': 53, 'municipio': 179, 'departamento': 32, 'categoria': 2, 'latitud': 1, 'propietario': 204, 'explotador': 156, 'longitud_pista': 1, 'ancho_pista': 2, 'pbmo': 38, 'orientacion': 1, 'elevacion': 244, 'resolucion': 255, 'fecha_construccion': 90, 'fecha_vigencia': 57, 'clase': 15, 'tipo': 5, 'numero_vuelos_origen': 204, 'gcd_departamento': 32, 'gcd_municipio': 181}


Con este analisis podemos concluir que ninguna de las columnas de la tabla posee alta cardinalidad, ya que como resultado del bloque anterior las columnas del tipo identificador unicos (`siglas` y `nombre`) son de las poseen alta cardinalidad, asi como el campo de `longitud`, son en donde se esperan muchos valores unicos al ser informacion unica de cada aeropuerto.


**Con estos dos analisis se puede determinar que nuestros datos poseen un buen nivel de Completitud, ya que no existen celdas dentro de los registros vacios y las cardinalidades indican baja relacion entre los elementos de la tabla.**

#### Consistencia:

A nivel de consistencia verifcaremos lo siguiente:

- Que exista una relacion 1 a 1 entrelos campos de `departamento` y `gcd_departamento`, y que ambas columnas estan relacionadas al proveer la misma informacion, solo con dos representaciones diferentes.
- Que exista una relacion 1 a 1 entrelos campos de `municipio` y `gcd_municipio`, y que ambas columnas estan relacionadas al proveer la misma informacion, solo con dos representaciones diferentes.
- Que los datos de `numero_vuelos_origen` correspondan a datos validos.

In [12]:
aero_dep_unicos = df_aeropuertos.select('departamento').distinct().count()
aero_gcddep_unicos = df_aeropuertos.select('gcd_departamento').distinct().count()

print('La cantidad de Departamentos que tienen aeropuerto segun nuestros registros son: ' + str(aero_dep_unicos) )
print('la cantidad de gcd_departamentos unicos son de : ' + str(aero_gcddep_unicos) )

La cantidad de Departamentos que tienen aeropuerto segun nuestros registros son: 32
la cantidad de gcd_departamentos unicos son de : 32


In [13]:
aero_mun_unicos = df_aeropuertos.select('municipio').distinct().count()
aero_gcdmun_unicos = df_aeropuertos.select('gcd_municipio').distinct().count()

print('La cantidad de Municipios que tienen aeropuerto segun nuestros registros son: ' + str(aero_mun_unicos) )
print('la cantidad de gcd_municipio unicos son de : ' + str(aero_gcdmun_unicos) )

La cantidad de Municipios que tienen aeropuerto segun nuestros registros son: 179
la cantidad de gcd_municipio unicos son de : 181


In [14]:
num_vuelos_negativos = df_aeropuertos.where(df_aeropuertos.numero_vuelos_origen <= 0).count()

print('La cantidad de registros con vuelos de origen con valor de 0 o menos es de : ' + str( num_vuelos_negativos))

La cantidad de registros con vuelos de origen con valor de 0 o menos es de : 346


**Con lo anterior se puede concluir que los datos poseen un buen nivel de consistencia a nivel de Departamente, pero a nivel de Municipio y numero de vuelos de origen existen algunos registros inconsistentes, por esto se le otorgara un valor de Consitencia Media**.

## Paso #4: Conclusiones del Entendimiento de los Datos:

En esta seccion nos enfocaremos en suministrar conclusiones a raiz de los pasos anteriores:

Como conclusion de la actividad de entendimiento de datos y analisis de calidad para la tabla de AeropuertosCopia, podemos decir que la misma necesita un poco de retrabajo, se encontraron muchos datos que si bien no estan vacios, corresponden a cadenas de caracteres vacios, se encontraron columnas que representan la misma informacion pero con representaciones de datos dierentes.

Se recomienda al negocio revisar las preguntas propuestas en la siguiente seccion para intentar mejorar la calidad de los datos y poder proseguir con el desarrollo del proyecto.

**Se puede concluir a nivel general que los datos extraido de esta tabla (AeropuertoCopia) posee un nivel de Calidad media, en donde se recomienda mejorar esta calidad para lograr un mayor porcentaje de exito y aprovechamiento por parte del negocio de este proyecto. No existen muchos datos inconsistentes, es por esto que se puede continuar con el proyecto con ciertas acalaracion.**

### Preguntas al Negocio:

En esta seccion enlistaremos las dudas que hayan surguido y las aclaraciones necesarias para poder continuar con el desarrollo del proyecto.

#### Tipos de Datos:

Algunos campos de la tabla poseen tipos de datos que no reflejan los tipos de datos que desean contener, la pregunta al negocio seria: **Se puede clarificar si estos comportamientos son esperados**.

Por ejemplo:
- latitud, posee el tipo de IntegerType que puede perder precisión de la ubicacion del aeropuerto.
- longitud, posee el tipo de StringType que no representa un numero lo que facilitaria su analisis correcto.
- pbmo , posee el tipo de StringType que no representa un numero lo que facilitaria su analisis correcto.
- elevacion, posee el tipo de StringType que no representa un numero lo que facilitaria su analisis correcto.

#### Datos posiblemente equivocados:

Algunos campos tienen datos que no hacen sentido en el contexto de la informacion presentada. **Podria el negocio aclarar lel significado de los siguiente:**

- Ancho de Pista igual a 0. (84 registros).
- Todos los registros tienen como longitud el mismo valor.
- Numero de vuelos cuyo origen es el aeropuerto en cuestion con valores negativos.
- Existen datos con equivocada codificacion UTF que necesitarian correccion para su propia lectura de los usuarios.

#### Posibles columnas a eliminar:

Con el analisis de los datos realizados existen columnas que podrian no aportar suficiente valor para ser considerad e el Analisis, **Podria el negocio confirmar que la eliminicion de estas es aceptable:**

- La longitud de pista siempre corresponde a 250 m, esto puede tratarse de un estandar nacional o internacional y podemos eliminar este parametro de los analisis.
- Los campos de `gcd_departamento` y `departamento`, asi como `gcd_municipio` y `municipio` pueden estar expresando la misma informacion, y podriamos eliminar alguno de estos en los analisis.
- El campo de `orientacion` esta compuesto en su totalidad por cadenas de caracteres vacias que hace que no tenga sentido conservarla.

#### Posibles datos faltantes:

Despues de hacer el analisis de completitud en el apartado de calidad en los datos, nos podemos cuestionar si no existiran datos faltantes en los registros entregados, esto dado a que existen una columna totalmente llena de cadenas de caracteres vacias (`orientacion`) pero ademas existen otras columnas con muchas cadenas de caracteres vacias en su contenido. **Podriael negocio revisar este punto y sugerir una forma de atender el problema**

## Paso #5: Clean Up
En esta seccion haremos limpieza de la sesion de Spark, ya que hemos concluido con lo esperado en la actividad.

In [15]:
spark.stop()